This test demonstrates the methods inside the dictionary object function as intended. The first test uses 64-bit complex numbers and the Woodbury form.

In [1]:
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr


fltrSz = (3,3)
fftSz = (8,8)
noc = 3
nof = 8
rho = 1.0
dtype = tf.complex64

AInv = fctr.dictionary_object2D(fltrSz=fltrSz,fftSz=fftSz,noc = noc,nof = nof,rho = rho,dtype=dtype)

#x = tf.keras.layers.Input(shape=fftSz + (noc,1))
# This throws error because the fft exploits symmetry, and this input is not in the same form.
w = tf.complex(tf.random.normal((100,) + fftSz + (noc,1),dtype=tf.float32),tf.random.normal((100,) + fftSz + (noc,1),dtype = tf.float32))
w = w + tf.math.conj(tf.reverse(w,axis=(2,3)))
w = w[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]

x = AInv.dhmul(w)

y = rho*x + AInv.dhmul(AInv.dmul(x))
z = AInv.qinv(y)
print(tf.math.reduce_max(tf.math.abs(x - z)))

tf.Tensor(0.00876935, shape=(), dtype=float32)


The second test uses 128-bit complex numbers and the Woodbury form.  Note the rounding error is substantially improved (by approximately 8 orders of magnitude).

In [1]:
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr
import transforms as transf
import math
import post_process_grad as ppg

fltrSz = (3,3)
fftSz = (8,8)
noc = 3
nof = 8
rho = 1.0
dtype = tf.complex128
nots = 100
novs = 100

AInv = fctr.dictionary_object2D_full(fltrSz=fltrSz,fftSz=fftSz,noc = noc,nof = nof,rho = rho,dtype=dtype)

wt = tf.complex(tf.random.normal((nots,) + fftSz + (noc,1),dtype=dtype.real_dtype),tf.random.normal((nots,) + fftSz + (noc,1),dtype = dtype.real_dtype))
wt = wt + tf.math.conj(tf.reverse(wt,axis=(2,3)))
wt = wt[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]
wv = tf.complex(tf.random.normal((novs,) + fftSz + (noc,1),dtype=dtype.real_dtype),tf.random.normal((novs,) + fftSz + (noc,1),dtype = dtype.real_dtype))
wv = wv + tf.math.conj(tf.reverse(wv,axis=(2,3)))
wv = wv[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]

B = transf.fft2d_inner(fftSz)(tf.random.normal((1,) + fltrSz + (noc,nof,),dtype=dtype.real_dtype))


xt = tf.linalg.matmul(B,wt,adjoint_a = True)
xv = tf.linalg.matmul(B,wv,adjoint_a = True)
idmat = tf.eye(nof,dtype=dtype,batch_shape = (int(tf.rank(B)) - 2)*(1,))
yt = tf.linalg.matmul(tf.linalg.inv(rho*idmat + tf.linalg.matmul(B,B,adjoint_a=True)),xt)
yv = tf.linalg.matmul(tf.linalg.inv(rho*idmat + tf.linalg.matmul(B,B,adjoint_a=True)),xv)

w = tf.keras.layers.Input(shape=(fftSz[0],math.floor(fftSz[1]/2.) + 1,) + (noc,1,),dtype=dtype)
x = tf.linalg.matmul(B,w,adjoint_a = True)
y1 = AInv.qinv(x)
y2 = fctr.QInv_auto(AInv.dhmul,rho)(x)

model1 = ppg.Model_record_grad(w,y2)
model1.compile(loss = tf.keras.losses.MSE,run_eagerly=True,optimizer=tf.keras.optimizers.SGD())
model1.fit(x=wt,y=yt,batch_size=10,epochs=8,shuffle=False,validation_data = (wv,yv))
model2 = ppg.Model_passenger(model1.gradient_record,w,y1)
model2.compile(loss = tf.keras.losses.MSE,run_eagerly=True,optimizer=tf.keras.optimizers.SGD())
model2.fit(x=wt,y=yt,batch_size=10,epochs=8,shuffle=False,validation_data = (wv,yv))
maxError = 0.
maxGrad = 0.
for grad1,grad2 in zip(model1.gradient_record,model2.gradient_record):
    for gradval1,gradval2 in zip(grad1,grad2):
        maxError = max(maxError,tf.math.reduce_max(tf.math.abs((gradval1 - gradval2)*tf.math.conj(gradval1 - gradval2))))
        maxGrad = max(maxGrad,tf.math.reduce_max(tf.math.abs(gradval1*tf.math.conj(gradval1))),tf.math.reduce_max(tf.math.abs(gradval2*tf.math.conj(gradval2))))
print(maxError)
print(maxGrad)

Epoch 1/8
10/10 [==============================] - 0s 14ms/step - loss: 76.8408 - val_loss: 80.4892
Epoch 2/8
10/10 [==============================] - 0s 14ms/step - loss: 76.1562 - val_loss: 79.7579
Epoch 3/8
10/10 [==============================] - 0s 15ms/step - loss: 75.4548 - val_loss: 79.0100
Epoch 4/8
10/10 [==============================] - 0s 14ms/step - loss: 74.7383 - val_loss: 78.2479
Epoch 5/8
10/10 [==============================] - 0s 17ms/step - loss: 74.0086 - val_loss: 77.4740
Epoch 6/8
10/10 [==============================] - 0s 16ms/step - loss: 73.2676 - val_loss: 76.6908
Epoch 7/8
10/10 [==============================] - 0s 15ms/step - loss: 72.5178 - val_loss: 75.9011
Epoch 8/8
10/10 [==============================] - 0s 16ms/step - loss: 71.7612 - val_loss: 75.1078
Epoch 1/8
10/10 [==============================] - 0s 22ms/step - loss: 76.8397 - val_loss: 80.4861
Epoch 2/8
10/10 [==============================] - 0s 20ms/step - loss: 76.1520 - val_loss: 79.7509


This test uses 64-bit complex numbers with standard formulation (no Woodbury).

In [1]:
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr


fltrSz = (3,3)
fftSz = (8,8)
noc = 8
nof = 3
rho = 1.0
dtype = tf.complex64

AInv = fctr.dictionary_object2D(fltrSz=fltrSz,fftSz=fftSz,noc = noc,nof = nof,rho = rho,dtype=dtype)

#x = tf.keras.layers.Input(shape=fftSz + (noc,1))
# This throws error because the fft exploits symmetry, and this input is not in the same form.
w = tf.complex(tf.random.normal((100,) + fftSz + (noc,1),dtype=tf.float32),tf.random.normal((100,) + fftSz + (noc,1),dtype = tf.float32))
w = w + tf.math.conj(tf.reverse(w,axis=(2,3)))
w = w[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]

x = AInv.dhmul(w)

y = rho*x + AInv.dhmul(AInv.dmul(x))
z = AInv.qinv(y)
print(tf.math.reduce_max(tf.math.abs(x - z)))

tf.Tensor(3.7482743e-05, shape=(), dtype=float32)


This final test uses 128-bit complex numbers with standard formulation (no woodbury).

In [1]:
import tensorflow as tf
import numpy as np
import matrix_decompositions_tf as fctr


fltrSz = (3,3)
fftSz = (8,8)
noc = 8
nof = 3
rho = 1.0
dtype = tf.complex128

AInv = fctr.dictionary_object2D(fltrSz=fltrSz,fftSz=fftSz,noc = noc,nof = nof,rho = rho,dtype=dtype)

#x = tf.keras.layers.Input(shape=fftSz + (noc,1))
# This throws error because the fft exploits symmetry, and this input is not in the same form.
w = tf.complex(tf.random.normal((100,) + fftSz + (noc,1),dtype=tf.float64),tf.random.normal((100,) + fftSz + (noc,1),dtype = tf.float64))
w = w + tf.math.conj(tf.reverse(w,axis=(2,3)))
w = w[slice(None),slice(None),slice(0,5,1),slice(None),slice(None)]

x = AInv.dhmul(w)

y = rho*x + AInv.dhmul(AInv.dmul(x))
z = AInv.qinv(y)
print(tf.math.reduce_max(tf.math.abs(x - z)))

tf.Tensor(1.1008039046578624e-13, shape=(), dtype=float64)
